In [2]:
import logging
import datetime
import time
#from sitemap_parser import SitemapParser
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, NoSuchAttributeException, InvalidSelectorException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import codecs
import re
import requests
import json
import pprint as pprint

In [ ]:
def chocolatewithgrace_crawler(self, path):
        """
        :param urls: Urls to parse.
        :param url_id: url_id from input table.
        :param website_id: site_id from input table.
        :param website_name: site_name.
        :return: Data points from website.
        """
        start_datetime = datetime.datetime.utcnow()
        path= 'C:/Users/Dell/Downloads/smart-cookie/chocolatewithgrace/'
        files = os.listdir(path)
        count = 0
        for i in range(len(files)):
            count = count + 1
            logger.info('Loop Count %d, DF index Count %d', count, len(self.buffer_data_df.index))
            files[i] = path + '/' + files[i]
            file = codecs.open(files[i], "r", "utf-8")
            soup = BeautifulSoup(file.read(), features="lxml")
            info_container = soup.select_one('div.wprm-recipe-container')
            if info_container:
                content_series = self.get_empty_content_series()
                content_series["crawl_timestamp"] = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
                content_series["url_id"] = self.site_doc['url_id']
                content_series["website_id"] = self.site_doc['site_id']
                content_series["website_name"] = self.site_doc['site_name']

                try:
                    title = soup.select_one('h2.wprm-recipe-name')
                    content_series['recipe_title'] = title.text
                except:
                    content_series['recipe_title'] = ""    

                try:
                    content_series['recipe_url'] = soup.select_one('meta[property="og:url"]')['content']
                except:
                    content_series['recipe_url'] = ""    

                try:
                    author_name = soup.select_one('h4.author-box__title').text
                    content_series['author_name'] = author_name
                except:
                    content_series['author_name']= ''
                    
                try:
                    try:
                        value = soup.select_one('span.wprm-recipe-servings.wprm-recipe-details.wprm-recipe-servings').text
                    except:
                        value = ""
                    try:
                        unit = soup.select_one('span.wprm-recipe-servings-unit.wprm-recipe-details-unit.wprm-block-text-normal').text
                    except:
                        unit = ""
                    serving_size = {'value': value, 'unit': unit}
                    content_series['serving_size'] = serving_size
                except:
                    content_series['serving_size'] = ""       

                try:
                    modified_date = soup.select_one('meta[property="article:modified_time"]')
                    content_series['modified_date'] = modified_date['content'].split('T')[0]
                except:
                    content_series['modified_date'] = ""

                try:
                    published_date = soup.select_one('meta[property="article:published_time"]')
                    content_series['published_date'] = published_date['content'].split('T')[0]
                except:
                    content_series['published_date'] = "" 
                
                try:
                    cuisine = info_container.select_one("span.wprm-recipe-cuisine")
                    if cuisine:
                        content_series['cuisine'] = cuisine.text
                except:
                    content_series['cuisine'] = ""
                
                try:
                    meal_course = soup.select_one("span.wprm-recipe-course")
                    if meal_course:
                        content_series['meal_course'] = meal_course.text
                except:
                    content_series['meal_course'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        preperation_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-prep_time.wprm-recipe-prep_time-minutes")
                        if preperation_time_minutes:
                            time_data['minutes'] = preperation_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        preperation_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-prep_time.wprm-recipe-prep_time-hours")
                        if preperation_time_hours:
                            time_data['hours'] = preperation_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        preperation_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-prep_time.wprm-recipe-prep_time-days")
                        if preperation_time_days:
                            time_data['days'] = preperation_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['preperation_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['preperation_time'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        cooking_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-cook_time.wprm-recipe-cook_time-minutes")
                        if cooking_time_minutes:
                            time_data['minutes'] = cooking_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        cooking_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-cook_time.wprm-recipe-cook_time-hours")
                        if cooking_time_hours:
                            time_data['hours'] = cooking_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        cooking_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-cook_time.wprm-recipe-cook_time-days")
                        if cooking_time_days:
                            time_data['days'] = cooking_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['cooking_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['cooking_time'] = ""

                time_data = {'hours': 0, 'minutes': 0, 'days': 0}
                try:
                    try:
                        total_time_minutes = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-total_time.wprm-recipe-total_time-minutes")
                        if total_time_minutes:
                            time_data['minutes'] = total_time_minutes.text
                    except:
                        time_data['minutes'] = "0"
                    try:
                        total_time_hours = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-total_time.wprm-recipe-total_time-hours")
                        if total_time_hours:
                            time_data['hours'] = total_time_hours.text
                    except:
                        time_data['hours'] = "0"
                    try:
                        total_time_days = info_container.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-total_time.wprm-recipe-total_time-days")
                        if total_time_days:
                            time_data['days'] = total_time_days.text
                    except:
                        time_data['days'] = "0"
                    content_series['total_time'] = int(time_data['minutes'])+int(time_data['hours'])*60+int(time_data['days'])*24*60
                except:
                    content_series['total_time'] = ""

               
                try:
                    image = soup.select_one('meta[property="og:image"]')['content']
                    images_dict = {'image':image, 'title': '', 'caption': ''}
                    content_series["img_urls"] = [images_dict]
                except:
                    content_series['img_urls'] = []
                
                try:
                    description = soup.select_one('meta[property="og:description"]')['content']
                    content_series['recipe_description'] = description
                except:
                    pass
                    
                ingredients_lst = []
                ingredient_container = soup.select_one('div.wprm-recipe-ingredients-container')
                if ingredient_container:
                    group_sections = ingredient_container.select('div.wprm-recipe-ingredient-group')
                    for section in group_sections:
                        ingredients = []
                        section_text = ''
                        section_tag = section.find('h4')
                        if section_tag:
                            section_text = section_tag.text
                        lists = section.select('ul>li.wprm-recipe-ingredient')
                        for li in lists:
                            ingredent_data = {'section': '', 'ingredients': []}
                            ingredient = li.find_all('span')
                            str1 = ""
                            for ele in ingredient:
                                if '▢' in ele.text:
                                    str1 = str1 + ' ' + ele.text.strip('▢')
                                else:
                                    str1 = str1 + ' ' + ele.text
                            ingredients.append(str1.strip())
                        ingredients_lst.append({'section': section_text, 'ingredients': ingredients})
                content_series["recipe_ingredient_list"] = ingredients_lst

                instructions_list = []
                title = ''
                instructions = soup.select('div.wprm-recipe-instruction-group>ul>li')
                instruction_lst = []
                for inst in instructions:
                    isnt_data = {'text': '', 'title': '', 'image': '', 'caption': ''}
                    isnt_data['text'] = inst.text
                    instruction_lst.append(isnt_data)
                instructions_list.append({'section': title, 'instructions': instruction_lst})
                content_series["recipe_instructions"] = instructions_list
                instruction_container = soup.select_one('div.wprm-recipe-instruction-group')
                if instruction_container:
                    content_series["recipe_instructions_html"] = str(instruction_container)
                
                self.parsed_data_df = self.parsed_data_df.append(content_series, ignore_index=True)
                self.buffer_data_df = self.buffer_data_df.append(content_series, ignore_index=True)
                if len(self.buffer_data_df.index) % 10 == 0:
                    self.buffer_data_df = self.buffer_data_df.replace({np.nan: None})
                    self.parsed_data_df = self.parsed_data_df.replace({np.nan: None})
                    recipe_content = self.buffer_data_df.to_dict('records')
                    recipe_urls = self.buffer_data_df[['recipe_url']].rename(
                        columns={'recipe_url': 'url'}).to_dict('records')
                    self._store_to_db(recipe_content, recipe_urls)
                    self.parsed_data_df.to_csv(self.site_doc['site_name'] + '.csv', index=False)
                    logger_stats.info('Start Time: %s, Total Dumps: %d, Elapsed Time: %s',
                                    start_datetime,
                                    len(self.buffer_data_df.index),
                                    datetime.datetime.utcnow() - start_datetime)
                    self.buffer_data_df.drop(self.buffer_data_df.index, inplace=True)
        if len(self.buffer_data_df.index):
            self.buffer_data_df = self.buffer_data_df.replace({np.nan: None})
            recipe_content = self.buffer_data_df.to_dict('records')
            recipe_url = self.buffer_data_df[['recipe_url']].rename(
                columns={'recipe_url': 'url'}).to_dict('records')
            self._store_to_db(recipe_content, recipe_url)
            self.parsed_data_df = self.parsed_data_df.replace({np.nan: None})
            self.parsed_data_df.to_csv(self.site_doc['site_name'] + '.csv', index=False)
        logger_stats.info('Start Time: %s, Total Dumps: %d, Elapsed Time: %s',
                        start_datetime,
                        len(self.buffer_data_df.index),
                        datetime.datetime.utcnow() - start_datetime)

In [2]:
url = 'https://chocolatewithgrace.com/bbq-chicken-pizza/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml') 

In [6]:
url = 'https://chocolatewithgrace.com/buttery-dinner-rolls/#wprm-recipe-container-3720'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml') 

In [7]:
title = soup.select_one('h2.wprm-recipe-name')
title.text

'Buttery Dinner Rolls'

In [27]:
recipe_url = soup.select_one('meta[property="og:url"]')['content']
recipe_url

'https://chocolatewithgrace.com/bbq-chicken-pizza/'

In [28]:
author_name = soup.select_one('h4.author-box__title').text
author_name

'Mary'

In [6]:
value = soup.select_one('span.wprm-recipe-servings.wprm-recipe-details.wprm-recipe-servings').text
value

'4'

In [7]:
unit = soup.select_one('span.wprm-recipe-servings-unit.wprm-recipe-details-unit.wprm-block-text-normal').text
unit

'servings'

In [8]:
serving_size = {'value': value, 'unit': unit}
serving_size

{'value': '4', 'unit': 'servings'}

In [9]:
modified_date = soup.select_one('meta[property="article:modified_time"]')
modified_date = modified_date['content'].split('T')[0]
modified_date

'2022-02-26'

In [10]:
published_date = soup.select_one('meta[property="article:published_time"]')
published_date = published_date['content'].split('T')[0]
published_date

'2022-03-10'

In [26]:
preperation_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-prep_time.wprm-recipe-prep_time-minutes")
preperation_time_minutes.text

'30'

In [27]:
cooking_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-cook_time.wprm-recipe-cook_time-minutes")
cooking_time_minutes.text

'15'

In [28]:
total_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-total_time.wprm-recipe-total_time-minutes")
total_time_minutes.text

'45'

In [ ]:
wprm-recipe-details-label wprm-block-text-bold wprm-recipe-time-label wprm-recipe-custom-time-label

In [ ]:
wprm-recipe-details-label wprm-block-text-bold wprm-recipe-time-label wprm-recipe-custom-time-label

In [ ]:
wprm-recipe-block-container wprm-recipe-block-container-inline wprm-block-text-normal wprm-recipe-time-container wprm-recipe-custom-time-container

In [ ]:
wprm-recipe-details wprm-recipe-details-hours wprm-recipe-custom_time wprm-recipe-custom_time-hours

In [22]:
#      Additional time
time_data = {'hours': 0, 'minutes': 0, 'days': 0}
try:

    try:

        custom_time_minutes = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-minutes wprm-recipe-custom_time wprm-recipe-custom_time-minutes']")
        if custom_time_minutes:
            time_data['minutes'] = custom_time_minutes.text
    except:
        time_data['minutes'] = "0"

    try:
        custom_time_hours = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-hours wprm-recipe-custom_time wprm-recipe-custom_time-hours']")
        if custom_time_hours:
            time_data['hours'] = custom_time_hours.text
    except:
        time_data['hours'] = "0"

    try:
        custom_time_days = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-days wprm-recipe-custom_time wprm-recipe-custom_time-days']")
        if custom_time_days:
            time_data['days'] = custom_time_days.text
    except:
        time_data['days'] = "0"

    additional_time = time_data

except:
    additional_time = ""

In [23]:
additional_time

{'hours': '0', 'minutes': '0', 'days': '0'}

In [44]:
#      Additional time
time_data = {'hours': 0, 'minutes': 0, 'days': 0}
try:

    try:

        custom_time_minutes = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-minutes.wprm-recipe-custom_time.wprm-recipe-custom_time-minutes")
        if custom_time_minutes:
            time_data['minutes'] = custom_time_minutes.text
    except:
        time_data['minutes'] = ""

    try:
        custom_time_hours = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-custom_time.wprm-recipe-custom_time-hours")
        if custom_time_hours:
            time_data['hours'] = custom_time_hours.text
    except:
        time_data['hours'] = ""

    try:
        custom_time_days = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-days.wprm-recipe-custom_time.wprm-recipe-custom_time-days")
        if custom_time_days:
            time_data['days'] = custom_time_days.text
    except:
        time_data['days'] = ""
    content_series['additional_time'] = time_data
except:
    content_series['additional_time'] = ""

In [45]:
additional_time

{'hours': '1', 'minutes': 0, 'days': 0}

In [ ]:
    #      Additional time
time_data = {'hours': 0, 'minutes': 0, 'days': 0}
try:
    try:
        custom_time_minutes = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-minutes wprm-recipe-custom_time wprm-recipe-custom_time-minutes']")
        if custom_time_minutes:
            time_data['minutes'] = custom_time_minutes.text
    except:
        time_data['minutes'] = ""
    try:
        custom_time_hours = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-hours wprm-recipe-custom_time wprm-recipe-custom_time-hours']")
        if custom_time_hours:
            time_data['hours'] = custom_time_hours.text
    except:
        time_data['hours'] = ""
    try:
        custom_time_days = self.find_element_by_xpath(
            "//span[@class='wprm-recipe-details wprm-recipe-details-days wprm-recipe-custom_time wprm-recipe-custom_time-days']")
        if custom_time_days:
            time_data['days'] = custom_time_days.text
    except:
        time_data['days'] = ""
    content_series['additional_time'] = time_data
except:
    content_series['additional_time'] = ""

In [ ]:
#      Additional time
time_data = {'hours': 0, 'minutes': 0, 'days': 0}
try:

    try:

        custom_time_minutes = soup.select_one("span.")
        if custom_time_minutes:
            time_data['minutes'] = custom_time_minutes.text
    except:
        time_data['minutes'] = ""

    try:
        custom_time_hours = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-custom_time.wprm-recipe-custom_time-hours")
        if custom_time_hours:
            time_data['hours'] = custom_time_hours.text
    except:
        time_data['hours'] = ""

    try:
        custom_time_days = soup.select_one("span.wprm-recipe-details.wprm-recipe-details-hours.wprm-recipe-custom_time.wprm-recipe-custom_time-hours")
        if custom_time_days:
            time_data['days'] = custom_time_days.text
    except:
        time_data['days'] = ""

    additional_time = time_data

except:
    additional_time = ""

In [14]:
image = soup.select_one('meta[property="og:image"]')['content']
images_dict = {'image':image, 'title': '', 'caption': ''}
img_url = [images_dict]
img_url

[{'image': 'https://chocolatewithgrace.com/wp-content/uploads/2022/02/CWG-BBQ-Chicken-French-Bread-Pizza-10-1-of-1-scaled.jpg',
  'title': '',
  'caption': ''}]

In [15]:
description = soup.select_one('meta[property="og:description"]')['content']
description

"BBQ Chicken French Bread Pizza is a pizza lover's dream come true! With just a few ingredients and about 20 minutes, you can have dinner on the table."

In [19]:
ingredients_lst = []
ingredient_container = soup.select_one('div.wprm-recipe-ingredients-container')
if ingredient_container:
    group_sections = ingredient_container.select('div.wprm-recipe-ingredient-group')
    for section in group_sections:
        ingredients = []
        section_text = ''
        section_tag = section.find('h4')
        if section_tag:
            section_text = section_tag.text
        lists = section.select('ul>li.wprm-recipe-ingredient')
        for li in lists:
            ingredent_data = {'section': '', 'ingredients': []}
            ingredient = li.find_all('span')
            str1 = ""
            for ele in ingredient:
                if '▢' in ele.text:
                    str1 = str1 + ' ' + ele.text.strip('▢')
                else:
                    str1 = str1 + ' ' + ele.text
            ingredients.append(str1.strip())
        ingredients_lst.append(
            {'section': section_text, 'ingredients': ingredients})

In [20]:
ingredients_lst

[{'section': '',
  'ingredients': ['1 loaf French or Italian bread',
   '2 tablespoons olive oil',
   '1/4 teaspoon kosher salt',
   '16 ounces mozzarella cheese',
   '1 1/2 cups barbecue sauce homemade or store-bought',
   '2 cups cooked chicken, shredded or chopped small',
   '2 tablespoons sliced green onions']}]

In [21]:
instructions_list = []
title = ''
instructions = soup.select('div.wprm-recipe-instruction-group>ul>li')
instruction_lst = []
for inst in instructions:
    isnt_data = {'text': '', 'title': '', 'image': '', 'caption': ''}
    isnt_data['text'] = inst.text
    instruction_lst.append(isnt_data)
instructions_list.append({'section': title, 'instructions': instruction_lst})
recipe_instructions = instructions_list
recipe_instructions

[{'section': '',
  'instructions': [{'text': 'Preheat the oven to 400°F. Slice the bread in half lengthwise and then cut each half into 3 pieces, as pictured. If the bread is thicker than about 1-inch, press down on each piece until it’s closer to that thickness.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'Place the pieces on a large baking sheet. Combine the olive oil and salt in a small bowl. Brush the sliced portion of each piece with the olive oil mixture.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'Top the bread with half of the cheese and place in the oven. Cook for 8-10 minutes until the cheese melts.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'While the cheese is melting, warm the sauce and the chicken in the microwave or on the stovetop. Remove the bread from the oven and spread the sauce evenly over the bread. Top with the chicken.',
    'title': '',
    'image': '',
    'caption': ''},
   {'text': 'Co

In [22]:
cuisine = soup.select_one("span.wprm-recipe-cuisine").text
cuisine

'American'

In [23]:
meal_course = soup.select_one("span.wprm-recipe-course")
meal_course.text

'Main Dishes'

In [24]:
des = soup.select_one('div.wprm-recipe-summary.wprm-block-text-normal')
des.text

"BBQ Chicken French Bread Pizza! This method creates cheesy pizzas with a crispy crust. It's homemade pizza that's fast enough for a quick lunch or casual dinner."

In [31]:
info_container = soup.select_one('div.wprm-recipe-container')
info_container

<div class="wprm-recipe-container" data-recipe-id="3720" data-servings="18" id="wprm-recipe-container-3720"><div class="wprm-recipe wprm-recipe-template-barefeet2"><div class="wprm-recipe-template-barefeet2-inner">
<div class="wprm-template-bf-header">
<div class="wprm-template-bf-header__image">
<div class="wprm-recipe-image wprm-block-image-normal"><img alt="Easy, Buttery Homemade Dinner Rolls can be made ahead and are perfect for Easter, Thanksgiving or Christmas Dinner." class="attachment-be_thumbnail_m size-be_thumbnail_m" data-lazy-sizes="(max-width: 346px) 100vw, 346px" data-lazy-src="https://chocolatewithgrace.com/wp-content/uploads/2018/03/Buttery-Dinner-Rolls-5-346x346.jpg" data-lazy-srcset="https://chocolatewithgrace.com/wp-content/uploads/2018/03/Buttery-Dinner-Rolls-5-346x346.jpg 346w, https://chocolatewithgrace.com/wp-content/uploads/2018/03/Buttery-Dinner-Rolls-5-500x500.jpg 500w, https://chocolatewithgrace.com/wp-content/uploads/2018/03/Buttery-Dinner-Rolls-5-150x150.jp